In [8]:
# !pip install tqdm # 진행바 설정

# DirectoryLoader + PyPDFLoader

pdf 출처 : https://www.koraia.org/default/mp5/sub3.php?sub=03
- 자료가 꽤 꾸준히 올라오기도하고, 퀄리티도 좋아서 ai관련해선 좋은 소스임

Langchain Document : https://python.langchain.com/docs/how_to/document_loader_directory/

In [72]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyPDFLoader

# Specify the directory containing your PDF files
pdf_folder_path = "/home/jun99/rag/pdfdata"

In [59]:
%%time
# Create a DirectoryLoader
loader = DirectoryLoader(pdf_folder_path, glob="**/*.pdf", loader_cls=PyPDFLoader,
                         show_progress=True) # 진행바 추가
# Load all PDF documents
documents = loader.load()

100%|██████████| 12/12 [00:52<00:00,  4.35s/it]

CPU times: user 46.8 s, sys: 1.13 s, total: 48 s
Wall time: 52.3 s


- PyPDFDirectoryLoader도 있긴한데, 확장성을 위해 DirectoryLoader사용

In [60]:
print(documents[-1])

page_content='책임 있는 AI를 위한 기업의 노력과 시사점
Corporate Efforts for Responsible AI and Implications
경기도 성남시 분당구 대왕판교로 712 번길 22 글로벌 R&D 연구동 (B) 4층
Global R&D Center 4F 22 Daewangpangyo-ro 712beon-gil, Bundang-gu, Seongnam-si, Gyeonggi-do
 
www.spri.kr  ISSN 2733-6336' metadata={'source': '/home/jun99/rag/pdfdata/[이슈리포트]_책임 있는 AI를 위한 기업의 노력과 시사점.pdf', 'page': 33}


- 그런데 일단 documents를 보면, page_content에 뭐랄까 metadata같은 내용들도 있어서 조금 그럼 

In [61]:
len(documents) # 딱 페이지 수만큼 나옴

818

#### What multithreading?
- 여러 파일들을 동시에 병렬적으로 다운받는 기능 : 효율적임! <- 대용량에서
- 근데 기본값으론 False인데, 안좋은점이 있는걸까? -> 안좋은건 내 노트북이였고


In [62]:
# %%time
# # 멀티스레드 활성화 : 병렬 로드 -> 하드 디스크 성능 이슈로 오히려 더 느려짐...쓰지말자 ㅠㅠ
# loader = DirectoryLoader(pdf_folder_path, glob="**/*.pdf", loader_cls=PyPDFLoader,
#                          show_progress=True, use_multithreading=True) 

# docs = loader.load()

앞으로 loader쓸 땐, 그냥 Directory로 정해서 쓰면 될 듯하다


범용성도 커서, 유지 & 보수도 하기 쉽다

# Text Splitter
- RecursiveCharacterTextSplitter많이 쓰는데, 보통 chunk_size(1000~1500)으로 잡고 overlap은 15~20%로 하는 게 일반적

- ref : https://github.com/langchain-ai/langchain/issues/2026

In [63]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Create a text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# Split the documents
split_docs = text_splitter.split_documents(documents)

In [64]:
type(split_docs), type(split_docs[0])

(list, langchain_core.documents.base.Document)

In [65]:
for i,split_doc in enumerate(split_docs[0:5]):
    print(f"========={i}번째 문서========================\n{split_doc}")

=========0번째 문서========================
page_content='ISSUE REPORT l 2024.08.19 IS-178
글로벌 초거대 AI 모델 현황 분석
(2020~2023년)
An Analysis on the Global Large-scale AI Model(2020-2023)
봉강호' metadata={'source': '/home/jun99/rag/pdfdata/(IS-178) 글로벌 초거대 AI 모델 현황 분석(2020~2023년)_SPRi_240819.pdf', 'page': 0}
=========1번째 문서========================
page_content='이 보고서는  ｢과학기술정보통신부  정보통신진흥기금 ｣에서 지원받아 제작한 것으로 
과학기술정보통신부의  공식의견과 다를 수 있습니다.
이 보고서의 내용은 연구진의 개인 견해이며, 본 보고서와 관련한 의문 사항 또는 수정·보완할 
필요가 있는 경우에는 아래 연락처로 연락해 주시기 바랍니다.
소프트웨어정책연구소  AI정책연구실
봉강호 선임연구원 bk91@spri.kr' metadata={'source': '/home/jun99/rag/pdfdata/(IS-178) 글로벌 초거대 AI 모델 현황 분석(2020~2023년)_SPRi_240819.pdf', 'page': 1}
=========2번째 문서========================
page_content='SPRi 이슈리포트 IS-178 글로벌 초거대 AI 모델 현황 분석(2020~2023년)
  CONTENT
    
Ⅰ. 서론 P.1
Ⅱ. 글로벌 초거대 AI 모델 현황 P.4
III. 요약 및 정책적 시사점 P.12
참고문헌 P.16' metadata={'source': '/home/jun99/rag/pdfdata/(IS-178) 글로벌 초거대 AI 모델 현황 분석(2020~2023년)_SPRi_240819.pdf', 'page': 2}
=========3번째 문서=========

# Embedding
https://wikidocs.net/233817

https://python.langchain.com/docs/integrations/platforms/huggingface/#huggingfacebgeembeddings

https://python.langchain.com/docs/integrations/text_embedding/bge_huggingface/

https://huggingface.co/spaces/mteb/leaderboard

자료들에 근거하여 bge임베딩 모델 사용 -> 제일 많이 기초모델로 사용됨
- 궁금한게 애초에 bge자체가 한국어 특화가 아니면, 현재 좋은 성능인 bge기반 다른 모델써도 되려나?

In [71]:
%%time
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-m3"
model_kwargs = {"device": "cpu"} # cuda or cpu
encode_kwargs = {"normalize_embeddings": True}
hf_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

KeyboardInterrupt: 

- embedding에 넣을 때 어떤 객체로 들어가야하지?
- 이거 때문에 한창 고생하긴했는데, 아직 잘 모르겠음

In [67]:
# 임베딩 과정을 확인하고 싶은데

# VectorDB
Chroma Vs FAISS VS Pinecone

### Chroma -> 요걸로 써야하나..?
https://github.com/chroma-core/chroma/tree/main/examples : chroma git

https://python.langchain.com/docs/integrations/vectorstores/chroma/ : langchain chroma

https://wikidocs.net/234094 : 테디노트 chroma
- 지속성 for 웹 어플리케이션 or 서비스 구축시
- 지속성 및 메타데이터 필터링 기능이 내장되어 있어 챗봇 대화 내역과 컨텍스트를 관리하는 데 유용하며, 인메모리 및 SQLite 등 다양한 배포 시나리오에 편리한 스토리지 옵션의 유연성을 제공

In [78]:
from langchain_chroma import Chroma

chroma_db = Chroma.from_documents(
    documents=split_docs,embedding=hf_embeddings,
    collection_name="chroma_db",
    persist_directory="./chroma_langchain_db"
)

ImportError: cannot import name 'PREPARE_METHODS' from 'pydantic._internal._std_types_schema' (/home/jun99/miniconda3/envs/jun/lib/python3.9/site-packages/pydantic/_internal/_std_types_schema.py)

In [77]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=hf_embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

ImportError: cannot import name 'PREPARE_METHODS' from 'pydantic._internal._std_types_schema' (/home/jun99/miniconda3/envs/jun/lib/python3.9/site-packages/pydantic/_internal/_std_types_schema.py)

### FAISS
https://python.langchain.com/docs/integrations/vectorstores/faiss/
- 주로 대용량 벡터를 처리할 때 사용 -> GPU가속 사용 가능
- 인덱싱 방법에 대한 세밀한 조정이 필요한 경우
- 근데, 따로 인덱싱 저장이 되니깐 괜찮지 않나?